# Data Cleaning and EDA


In [32]:
# Import the required libraries
import pandas as pd
import numpy as np
import re
 

In [93]:
# Functions for data cleaning
usa_states_fullname_regex = '(ALABAMA|ALASKA|ARIZONA|ARKANSAS|CALIFORNIA|COLORADO|CONNECTICUT|DELAWARE|FLORIDA|GEORGIA|HAWAII|' \
                            'IDAHO|ILLINOIS|INDIANA|IOWA|KANSAS|KENTUCKY|LOUISIANA|MAINE|MARYLAND|MASSACHUSETTS|MICHIGAN|MINNESOTA|MISSISSIPPI|MISSOURI|MONTANA|'\
                            'NEBRASKA|NEVADA|NEW\sHAMPSHIRE|NEWSJERSEY|NEW\sMEXICO|NEW\sYORK|NORTH\sCAROLINA|NORTH\sDAKOTA|OHIO|OKLAHOMA|OREGON|PENNSYLVANIA|RHODE\sISLAND|'\
                            'SOUTH\sCAROLINA|SOUTH\sDAKOTA|TENNESSEE|TEXAS|UTAH|VERMONT|VIRGINIA|WASHINGTON|WEST\sVIRGINIA|WISCONSIN|WYOMING|USA)'


usa_states_regex = ',\s{1}(A[KLRZ]|C[AOT]|D[CE]|FL|GA|HI|I[ADLN]|K[SY]|LA|M[ADEINOST]|N[CDEHJMVY]|O[HKR]|P[AR]|RI|S[CD]|T[NX]|UT|V[AIT]|W[AIVY])'

#  Functions for Duplicate checks 
def get_exact_dups(df):
    '''
    Returns duplicates
    '''
    dups = df[df.duplicated()]
    return dups

def get_tweet_dups(df, col_names):
    '''
    Returns duplicates based on given column name
    '''
    dups = df[df.duplicated(subset=col_names)]
    return dups

def get_is_us_loc(loc_string):
    matches_abbrev = bool(re.search(usa_states_regex, loc_string.upper()))
    if not matches_abbrev:
        matches_full_name = bool(re.search(usa_states_fullname_regex, loc_string.upper())) 
    return (matches_abbrev or matches_full_name)
    

In [19]:
external_data = pd.read_csv('../external_data/tweets_raw.csv')
print(external_data.shape)
print(external_data.info())

(202645, 8)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 202645 entries, 0 to 202644
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   Unnamed: 0     202645 non-null  int64 
 1   Unnamed: 0.1   202645 non-null  int64 
 2   Content        202645 non-null  object
 3   Location       155123 non-null  object
 4   Username       202645 non-null  object
 5   Retweet-Count  202645 non-null  int64 
 6   Favorites      202645 non-null  int64 
 7   Created at     202645 non-null  object
dtypes: int64(4), object(4)
memory usage: 12.4+ MB
None


In [23]:
external_data.head(5)

,Unnamed: 0,Unnamed: 0.1,Content,Location,Username,Retweet-Count,Favorites,Created at
0,0,0,innovate an innovative approach #quoteoftheday...,NaN,PaulBillygraha1,0,0,2020-08-02 04:56:27
1,1,1,The pandemic is raising concerns about how tee...,Worldwide,IAM__Network,0,0,2020-08-02 04:49:43
2,2,2,STI: Staying Education-ready in the New Normal...,Worldwide,IAM__Network,0,0,2020-08-02 04:32:36
3,3,3,Digital Learning Through Digital RCRT\n.\n.\nR...,NaN,digitalrcrt,0,0,2020-08-02 04:30:12
4,4,4,"Upswing Classroom: Out and Out Virtual School,...",India,etr_in,1,0,2020-08-02 04:00:21


In [25]:

# duplicate content?  Yes...drop it!
tweet_dups = get_tweet_dups(external_data, ['Content'])
print(len(tweet_dups))
print(tweet_dups['Content'].unique().size)

20660
15967


In [46]:
# Drop the rows with Location NaN
external_data.dropna(subset=['Location'], inplace=True)
external_data.drop_duplicates(subset=['Content'], inplace=True)
print(external_data.shape)


(139294, 9)


In [94]:
external_data['is_us_loc'] = external_data.apply(lambda row: get_is_us_loc(row['Location']), axis=1)

In [98]:
us_only_data = external_data[external_data['is_us_loc'] == True]
us_only_data.shape

(64454, 9)

In [101]:
us_only_data.tail()

,Unnamed: 0,Unnamed: 0.1,Content,Location,Username,Retweet-Count,Favorites,Created at,is_us_loc
202634,202634,2676,Told my kids this was real #virtuallearning #s...,"Knoxville, TN",donnyr71,0,5,2020-08-06 00:47:55,True
202635,202635,2677,Attention ALL students transitioning to colleg...,"Atlanta, GA",TeeTaylorMade,0,1,2020-08-06 00:47:24,True
202636,202636,2678,Attention ALL students transitioning to colleg...,"Chicago, IL",EZTAYLORFDN,0,1,2020-08-06 00:47:09,True
202637,202637,2679,I work with the best English 10 teacher in the...,"Roanoke, VA",abchitwood,0,2,2020-08-06 00:42:25,True
202639,202639,2681,Join us on August 21st at 12 pm CDT for Virtua...,"Tulsa, OK",LindaJatJCG,0,0,2020-08-06 00:30:58,True


In [102]:
us_only_data.drop(['is_us_loc'], axis=1)

,Unnamed: 0,Unnamed: 0.1,Content,Location,Username,Retweet-Count,Favorites,Created at
9,9,9,“Instructional Considerations for the 2020-21 ...,"Illinois, USA",Erik_Youngman,0,2,2020-08-02 00:10:26
10,10,10,With all the uncertainty of what September wil...,"Lyndhurst, NJ",Renee_LoBue,0,0,2020-08-01 23:57:31
11,11,11,Check this out on Wakelet - Digital learning a...,"Cary, NC",SupriyaVasu,0,0,2020-08-01 23:20:38
12,12,12,Happy Friendship Day!\n#rdnums #nagaland #kohi...,"Kohima, India",rdnums,2,1,2020-08-01 23:17:09
13,13,13,Beat the summer heat with over 400 cool games ...,"Providence, RI",ABCyaGames,0,2,2020-08-01 23:00:00
...,...,...,...,...,...,...,...,...
202634,202634,2676,Told my kids this was real #virtuallearning #s...,"Knoxville, TN",donnyr71,0,5,2020-08-06 00:47:55
202635,202635,2677,Attention ALL students transitioning to colleg...,"Atlanta, GA",TeeTaylorMade,0,1,2020-08-06 00:47:24
202636,202636,2678,Attention ALL students transitioning to colleg...,"Chicago, IL",EZTAYLORFDN,0,1,2020-08-06 00:47:09
202637,202637,2679,I work with the best English 10 teacher in the...,"Roanoke, VA",abchitwood,0,2,2020-08-06 00:42:25


In [103]:
us_only_data.to_csv('../data/us_only_external_data_tweets.csv')